# Loading a Zero ShotClassification Model from Hugging Face into Elasticsearch

This code will show you how to load a Zero Shot classification model into elasticsearch.

First we will configure our python environment

1. Set up our python environment
2. Load the chosed model from HF into Elasticsearch
3. Deploy and start the model on ML nodes
4. Test using the  model


### Elastic version support
Requires Elastic version 8.0+ with a platinum or enterprise license (or trial license)

You can set up a [free trial elasticsearch Deployment in Elastic Cloud](https://cloud.elastic.co/registration).

---
---
# Setup
---
---
This section will set up the python environment with the required libraries

## Install and import required python libraries

Elastic uses the [eland python library](https://github.com/elastic/eland) to download modesl from Hugging Face hub and load them into elasticsearch

In [1]:
pip install eland

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 157.9/157.9 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.6/412.6 kB 28.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.8/59.8 kB 4.5 MB/s eta 0:00:00


In [2]:
pip install elasticsearch

In [3]:
pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 20.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 38.9 MB/s eta 0:00:00


In [4]:
pip install sentence_transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.3 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125923 sha256=1d3d320129af35f9afe33bdc2a45dcc429f96674fc2b11f396dad63937275105
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [5]:
pip install torch==1.11

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 750.6/750.6 MB 1.9 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.1.0+cu118
    Uninstalling torch-2.1.0+cu118:
      Successfully uninstalled torch-2.1.0+cu118
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.1.0+cu118 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchdata 0.7.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchtext 0.16.0 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.
torchvision 0.16.0+cu118 requires torch==2.1.0, but you have torch 1.11.0 which is incompatible.


In [6]:
from pathlib import Path
from eland.ml.pytorch import PyTorchModel
from eland.ml.pytorch.transformers import TransformerModel
from elasticsearch import Elasticsearch
from elasticsearch.client import MlClient

/usr/local/lib/python3.10/dist-packages/eland/ml/_optional.py:116: UserWarning: Eland requires version '1.3' or newer of 'sklearn' (version '1.2.2' currently installed). Use pip or conda to update sklearn.
  warnings.warn(msg, UserWarning)
<ipython-input-6-ed653d9ea084>:5: DeprecationWarning: Importing from the 'elasticsearch.client' module is deprecated. Instead use 'elasticsearch' module for importing the client.
  from elasticsearch.client import MlClient


## Configure elasticsearch authentication.
The recommended authentication approach is using the [Elastic Cloud ID](https://www.elastic.co/guide/en/cloud/current/ec-cloud-id.html) and a [cluster level API key](https://www.elastic.co/guide/en/kibana/current/api-keys.html)

You can use any method you wish to set the required credentials. We are using getpass in this example to prompt for credentials to avoide storing them in github.

In [7]:
import getpass

In [8]:
es_cloud_id = getpass.getpass('Enter Elastic Cloud ID:  ')
#es_api_id = getpass.getpass('Enter cluster API key ID:  ')
es_api_key = getpass.getpass('Enter cluster API key:  ')

Enter Elastic Cloud ID:  ··········
Enter cluster API key:  ··········


## Connect to Elastic Cloud

In [9]:
es = Elasticsearch(cloud_id=es_cloud_id,
                   api_key=es_api_key
                   )
es.info() # should return cluster info

ObjectApiResponse({'name': 'instance-0000000058', 'cluster_name': 'a7bf48bf42ad403ab45dd6b90b860f85', 'cluster_uuid': 'gEbjuhUOSyCVzG4Gz2SQ2w', 'version': {'number': '8.11.0', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'd9ec3fa628c7b0ba3d25692e277ba26814820b20', 'build_date': '2023-11-04T10:04:57.184859352Z', 'build_snapshot': False, 'lucene_version': '9.8.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

---
---
# Loading a model from Hugging Face model hub
---
---

The Elastic Stack machine learning features support transformer models that conform to the standard BERT model interface and use the WordPiece tokenization algorithm.

A current list of supported architectures can be viewed in the
[Elastic NLP Model Support Docs](https://www.elastic.co/guide/en/machine-learning/current/ml-nlp-model-ref.html)


## Download Classification model from Hugging Face

For this example we will be using the [valhalla/distilbart-mnli-12-6](https://huggingface.co/valhalla/distilbart-mnli-12-6) model


### Download the Model
Here we specify the model id from Hugging Face. The easiest way to get this id is clicking the copy the model name icon next to the name on the model page.

When calling `TransformerModel` you specify the HF model id and the task type. You can try specifying `auto` and eland will attempt to determine the correct type from info in the model config. This is not always possible so a list of specific `task_type` values can be viewed in the following code:
[Supported values](https://github.com/elastic/eland/blob/15a300728876022b206161d71055c67b500a0192/eland/ml/pytorch/transformers.py#*L41*)

In [23]:
hf_model_id='valhalla/distilbart-mnli-12-6'
tm = TransformerModel(model_id=hf_model_id, task_type="zero_shot_classification")

### Set and confirm the model ID
To make the name compatible with elasticsearch, the '/' is replaced with '__'



In [24]:
es_model_id = tm.elasticsearch_model_id()
es_model_id

'valhalla__distilbart-mnli-12-6'

## Export the model in a TorchScrpt representation which Elasticsearch uses

In [25]:
tmp_path = "models"
Path(tmp_path).mkdir(parents=True, exist_ok=True)
model_path, config, vocab_path = tm.save(tmp_path)

## Load the model into Elasticsearch
Model should not already exist in elasticsearch

In [26]:
ptm = PyTorchModel(es, es_model_id)
ptm.import_model(model_path=model_path, config_path=None, vocab_path=vocab_path, config=config)

  0%|          | 0/1170 [00:00<?, ? parts/s]

---
---
# Starting the Model
---
---

## View information about the model
This is not required but can be handy to get a model overivew

In [27]:
m = MlClient.get_trained_models(es, model_id=es_model_id)
m.body

{'count': 1,
 'trained_model_configs': [{'model_id': 'valhalla__distilbart-mnli-12-6',
   'model_type': 'pytorch',
   'created_by': 'api_user',
   'version': '11.0.0',
   'create_time': 1700235097408,
   'model_size_bytes': 0,
   'estimated_operations': 0,
   'license_level': 'platinum',
   'description': "Model valhalla/distilbart-mnli-12-6 for task type 'zero_shot_classification'",
   'tags': [],
   'metadata': {'per_allocation_memory_bytes': 4824277356,
    'per_deployment_memory_bytes': 1638027276},
   'input': {'field_names': ['text_field']},
   'inference_config': {'zero_shot_classification': {'vocabulary': {'index': '.ml-inference-native-000002'},
     'tokenization': {'roberta': {'do_lower_case': False,
       'with_special_tokens': True,
       'max_sequence_length': 1024,
       'truncate': 'first',
       'span': -1,
       'add_prefix_space': False}},
     'classification_labels': ['contradiction', 'neutral', 'entailment'],
     'multi_label': False,
     'hypothesis_templa

## Deploy the model
This will load the model on the ML nodes and start the process(es) making it available for the NLP task

In [ ]:
s = MlClient.start_trained_model_deployment(es, model_id=es_model_id)
s.body

## Verify the model started without issue

In [36]:
stats = MlClient.get_trained_models_stats(es, model_id=es_model_id)
stats.body['trained_model_stats'][0]['deployment_stats']['nodes'][0]['routing_state']

{'routing_state': 'started'}

---
---
# Generating Entities
---
---
There are two main ways NLP modles, and the entities they generate, are used in elasticsearch

1. Calling the `_infer` endpoint with an input string and getting the entities back in the response. The entities are not stored in elastic (directly) and can be used for tasks such as enhancing follow up queries.
2. Passing a field from a document on ingest to a model with an ingest pipeline. The resulting entites are stored along with the document when it is indexed in elasticsearch.

For this example, we will be taking the first approach calling `_infer` to test the  model works with our data.

# Setting up the index in elasticsearch to store vectors



## Generate Vector for Query

Before we can run a kNN query, we need to convert our query string to a vector.

Creating a sample string

In [34]:
docs =  [
    {
      "text_field": "I have been traveling a lot for work this year, I'm happy to have the next week off."
    },
        {
      "text_field": "My house is leaking water and the dog ran away."
    },
        {
      "text_field": "It's pizza night!"
    }
  ]

In [40]:

inference_config = {
    "zero_shot_classification": {
      "labels": [
        "vacation",
        "sad",
        "work",
        "location",
        "travel",
        "monkey",
        "food",
        "space ship"
      ],
      "multi_label": True
    }
  }

We call the `_infer` endpoint supplying the model_id and the doc[s] we want generate entities for.

In [41]:
response = MlClient.infer_trained_model(es, model_id=es_model_id, docs=docs, inference_config=inference_config)

The vector for the first doc can be accessed in the response dict as shown below

In [42]:
doc_0_vector = response['inference_results']
doc_0_vector

[{'predicted_value': 'travel',
  'top_classes': [{'class_name': 'travel',
    'class_probability': 0.9988278261336632,
    'class_score': 0.9988278261336632},
   {'class_name': 'work',
    'class_probability': 0.9918028571732135,
    'class_score': 0.9918028571732135},
   {'class_name': 'location',
    'class_probability': 0.9190888847729426,
    'class_score': 0.9190888847729426},
   {'class_name': 'sad',
    'class_probability': 0.6780602507660834,
    'class_score': 0.6780602507660834},
   {'class_name': 'vacation',
    'class_probability': 0.5218332211782771,
    'class_score': 0.5218332211782771},
   {'class_name': 'food',
    'class_probability': 0.36727615313295,
    'class_score': 0.36727615313295},
   {'class_name': 'space ship',
    'class_probability': 0.25075667144572533,
    'class_score': 0.25075667144572533},
   {'class_name': 'monkey',
    'class_probability': 0.2203642171624536,
    'class_score': 0.2203642171624536}],
  'prediction_probability': 0.9988278261336632},
 